<a href="https://colab.research.google.com/github/agusnieto77/taller_pict_2019/blob/main/Colabs/ACEP_funciones_desarrollo_I.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejercicio 3

In [ ]:
install.packages("ACEP")

In [ ]:
require(ACEP)

In [ ]:
dic_conf <- acep_load_base(acep_diccionarios$dicc_confl_sismos)
print(dic_conf)

In [ ]:
texto <-     "
    La UTA inició un paro de colectivos por tiempo indeterminado
    El gremio que nuclea a los choferes lo resolvió por un atraso salarial. 
    La medida de fuerza comenzó a partir de las 18.
    Por un atraso salarial, la Unión Tranviarios Automotor (UTA) inició un paro 
    por tiempo indeterminado a partir de las 18 de este lunes, según 
    confirmaron fuentes del gremio que nuclea a los choferes.
    "

In [ ]:
texto_limpio <- acep_clean(texto)
texto_limpio

In [ ]:
acep_men(texto_limpio, dic_conf)

In [ ]:
acep_men(texto, dic_conf)